###**Import Libraries**

In [0]:
#!!pip uninstall tensorflow
#!pip install tensorflow==2.1.0
#!pip install -U gensim

In [5]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
PROJECT_DIR = "/content/drive/My Drive/MachineLearning/Capstone/"
DATA_FILE_NAME = 'Data/Input Data Synthetic.xlsx'
CLEANSED_FILE_DIR = 'Data/processed/'

In [7]:
#import basic libraies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [8]:
import keras
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, Concatenate, Dense, LSTM, Embedding, Dropout, Activation, SpatialDropout1D
from tensorflow.keras.layers import Bidirectional, Flatten, BatchNormalization, Conv1D, MaxPooling1D, SimpleRNN
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import tensorflow.keras.regularizers as regularizers
import tensorflow.keras.optimizers as optimizers
#from tensorflow.keras.layers.merge import concatenate
from keras.utils.vis_utils import plot_model


Using TensorFlow backend.


###**Load Document**

In [0]:
def load_doc(filename):
  if(filename.endswith('.xlsx')):
    data_df = pd.read_excel(filename, lines=True)
  elif(filename.endswith('.csv')):
    data_df = pd.read_csv(filename, keep_default_na = False)
  return data_df

In [0]:
def encode_target(target):
  le = LabelEncoder()
  return le.fit_transform(target), le

def decode_prediction(pred, encoder):
  return encoder.inverse_transform(pred) 

In [11]:
cleansed_data_df = load_doc(PROJECT_DIR+CLEANSED_FILE_DIR+'cleansed_data.csv')
print('shape of Data : ', cleansed_data_df.shape)

shape of Data :  (5419, 5)


In [12]:
cleansed_data_df.describe()

,short_description,description,combined_desc,caller,assignment_group
count,5419,5419,5419,5419,5419
unique,3976,5419,5419,2351,52
top,,install project,password locked password manager password lock...,rbozivdq gmlhrtvp,GRP_0
freq,306,1,1,46,2619


In [13]:
cleansed_data_df.head()

,short_description,description,combined_desc,caller,assignment_group
0,login issue,user manager checked user ad reset password ad...,login issue user manager checked user ad reset...,spxjnwir pjlcoqds,GRP_0
1,outlook,received team not outlook calendar somebody ad...,outlook received team not outlook calendar som...,hmjdrvpb komuaywn,GRP_0
2,cant log,received cannot log,cant log received cannot log,eylqgodm ybqkwiam,GRP_0
3,unable access page,unable access page,unable access page unable access page,xbkucsvz gcpydteq,GRP_0
4,error,error,error error,owlgqjme qhcozdfx,GRP_0


In [14]:
cleansed_data_df['assignment_group'].value_counts()

GRP_0         2619
GRP_2          218
GRP_12         196
GRP_19         195
GRP_3          190
GRP_8          137
GRP_13         136
GRP_14         110
GRP_25         103
GRP_29          93
GRP_16          83
GRP_18          82
GRP_24          81
GRP_4           78
GRP_10          75
GRP_33          71
GRP_9           66
GRP_7           65
GRP_6           58
GRP_26          54
GRP_COMMON      52
GRP_34          50
GRP_40          45
GRP_28          42
GRP_41          39
GRP_15          38
GRP_20          36
GRP_22          31
GRP_31          30
GRP_11          30
GRP_21          28
GRP_45          27
GRP_42          25
GRP_23          24
GRP_62          23
GRP_5           21
GRP_1           17
GRP_37          16
GRP_27          15
GRP_50          13
GRP_44          13
GRP_47          12
GRP_39          11
GRP_65          10
GRP_36          10
GRP_53           9
GRP_52           9
GRP_55           8
GRP_51           7
GRP_59           6
GRP_46           6
GRP_30           6
Name: assign

In [15]:
cleansed_data_df['description_length'] = cleansed_data_df['description'].str.len()
desc_max_length = cleansed_data_df['description_length'].max()
maxlen_desc_idx = cleansed_data_df['description_length'].idxmax()
print('Desc with max length ({}) at index {}:\n{}'.format(desc_max_length,maxlen_desc_idx,cleansed_data_df['description'].loc[maxlen_desc_idx]))

Desc with max length (2900) at index 2662:
source system user location unknown status field user no event log incident overview seeing device generating server response sinkhole set probable infected asset traffic not blocked port port device host likely infected return traffic likely visit domain give false information order prevent use domain address resolution sinkhole traffic possible indicator infected computer reaching controller taken law enforcement research organization part mitigation effort traffic sinkhole activity administrator may curious particular domain browse signature repeated sinkhole however clear indication infection incident via high priority ticket per default would like us handle differently future see handling let us know either corresponding us via ticket ticket back soc calling us ticket sinkhole domain explicit notification via medium priority ticket no phone call sinkhole domain directly portal no explicit notification available portal sincerely soc techni

###**Train & Test Data:**

In [0]:
#X = cleansed_data_df['description']
X = cleansed_data_df['combined_desc']
y = cleansed_data_df.assignment_group
y, le = encode_target(y)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.1, stratify=y, random_state = 10)

In [18]:
print('Shape of X_train : ',X_train.shape)
print('Shape of X_test : ',X_test.shape)
print('Shape of y_train : ',y_train.shape)
print('Shape of y_test : ',y_test.shape)

Shape of X_train :  (4877,)
Shape of X_test :  (542,)
Shape of y_train :  (4877,)
Shape of y_test :  (542,)


In [0]:
#pd.DataFrame(data = y_test.assignment_group.value_counts())

###**TF-IDF Vectors as features :**

In [0]:
unigram_vocab = load_doc(PROJECT_DIR+CLEANSED_FILE_DIR+'tfidf_based_unigram_tokens.csv')
bigram_vocab = load_doc(PROJECT_DIR+CLEANSED_FILE_DIR+'tfidf_based_bigram_tokens.csv')
trigram_vocab = load_doc(PROJECT_DIR+CLEANSED_FILE_DIR+'tfidf_based_trigram_tokens.csv')

In [21]:
print('unigram vocab : ', unigram_vocab.shape)
print('bigram vocab : ', bigram_vocab.shape)
print('trigram vocab : ', trigram_vocab.shape)

unigram vocab :  (2659, 2)
bigram vocab :  (26692, 2)
trigram vocab :  (36155, 2)


In [0]:
unigram_bigram_vocab = unigram_vocab.append(bigram_vocab)
unigram_bigram_trigram_vocab = unigram_bigram_vocab.append(trigram_vocab)

In [23]:
print('unigram vocab : ', unigram_vocab.shape)
print('unigram + bigram vocab : ', unigram_bigram_vocab.shape)
print('unigram + bigram + trigram vocab : ', unigram_bigram_trigram_vocab.shape)

unigram vocab :  (2659, 2)
unigram + bigram vocab :  (29351, 2)
unigram + bigram + trigram vocab :  (65506, 2)


In [0]:
unigram_max_features = 5000
bigram_max_features = 20000
trigram_max_features = 40000

maxlen = cleansed_data_df['description_length'].max()
embedding_size = 200

In [25]:
unigram_tokenizer = TfidfVectorizer(ngram_range=(1,1), max_features = unigram_max_features, vocabulary=unigram_vocab['words']).fit(X)
X_train_unigram_seq = unigram_tokenizer.transform(X_train)
X_test_unigram_seq = unigram_tokenizer.transform(X_test)
print('Vocabulary size of unigram :', len(unigram_tokenizer.vocabulary_))

bigram_tokenizer = TfidfVectorizer(ngram_range=(1,2), max_features = bigram_max_features, vocabulary=unigram_bigram_vocab['words']).fit(X)
X_train_bigram_seq = bigram_tokenizer.transform(X_train)
X_test_bigram_seq = bigram_tokenizer.transform(X_test)
print('Vocabulary size of bigram :', len(bigram_tokenizer.vocabulary_))

trigram_tokenizer = TfidfVectorizer(ngram_range=(1,3), max_features = trigram_max_features, vocabulary=unigram_bigram_trigram_vocab['words']).fit(X)
X_train_trigram_seq = trigram_tokenizer.transform(X_train)
X_test_trigram_seq = trigram_tokenizer.transform(X_test)
print('Vocabulary size of trigram :', len(trigram_tokenizer.vocabulary_))

Vocabulary size of unigram : 2659
Vocabulary size of bigram : 29351
Vocabulary size of trigram : 65506


In [26]:
print('Shape of X_train_unigram_seq : ',X_train_unigram_seq.shape)
print('Shape of X_test_unigram_seq : ',X_test_unigram_seq.shape)
print('Shape of X_train_bigram_seq : ',X_train_bigram_seq.shape)
print('Shape of X_test_bigram_seq : ',X_test_bigram_seq.shape)
print('Shape of X_train_trigram_seq : ',X_train_trigram_seq.shape)
print('Shape of X_test_trigram_seq : ',X_test_trigram_seq.shape)

Shape of X_train_unigram_seq :  (4877, 2659)
Shape of X_test_unigram_seq :  (542, 2659)
Shape of X_train_bigram_seq :  (4877, 29351)
Shape of X_test_bigram_seq :  (542, 29351)
Shape of X_train_trigram_seq :  (4877, 65506)
Shape of X_test_trigram_seq :  (542, 65506)


###**Machine Learning Models :**

#####Pre-Requiste

In [0]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm, neighbors
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers, regularizers
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [0]:
def train_model(model, feature_train, target_train, feature_validate, target_validate):
    # fit the training dataset on the classifier
    fitted_model = model.fit(feature_train, target_train)
    # predict the labels on validation dataset
    predictions = model.predict(feature_validate)
    return (fitted_model.score(feature_train, target_train)*100).round(3),(metrics.accuracy_score(predictions, target_validate)*100).round(3)

In [0]:
accuracy_text = '| Train : {} ~~ Test: {} |'

In [0]:
accuracy_df = pd.DataFrame(columns = ['Model' , 'unigram-tfidf Accuracy', 'bigram-tfidf Accuracy' , 'trigram-tfidf Accuracy']) 

In [0]:
accuracy_df.shape

(0, 4)

####**Naive Bayes :**

In [0]:
# Naive Bayes on tfidf Vectors
train_acc1, test_acc1 = train_model(naive_bayes.MultinomialNB(), X_train_unigram_seq, y_train, X_test_unigram_seq, y_test)
train_acc2, test_acc2 = train_model(naive_bayes.MultinomialNB(), X_train_bigram_seq,  y_train, X_test_bigram_seq,  y_test)
train_acc3, test_acc3 = train_model(naive_bayes.MultinomialNB(), X_train_trigram_seq, y_train, X_test_trigram_seq, y_test)

accuracy_df = accuracy_df.append(pd.Series(['Naive Bayes', accuracy_text.format(train_acc1, test_acc1),
                                            accuracy_text.format(train_acc2, test_acc2),
                                            accuracy_text.format(train_acc3, test_acc3),],
                                           index=accuracy_df.columns), ignore_index=True)
accuracy_df

,Model,unigram-tfidf Accuracy,bigram-tfidf Accuracy,trigram-tfidf Accuracy
0,Naive Bayes,| Train : 51.323 ~~ Test: 51.292 |,| Train : 50.072 ~~ Test: 50.0 |,| Train : 49.846 ~~ Test: 49.815 |


####**Linear Classifier :**

In [0]:
# Logistic Regression on tfidf Vectors
train_acc1, test_acc1 = train_model(linear_model.LogisticRegression(), X_train_unigram_seq, y_train, X_test_unigram_seq, y_test)
train_acc2, test_acc2 = train_model(linear_model.LogisticRegression(), X_train_bigram_seq,  y_train, X_test_bigram_seq,  y_test)
train_acc3, test_acc3 = train_model(linear_model.LogisticRegression(), X_train_trigram_seq, y_train, X_test_trigram_seq, y_test)

accuracy_df = accuracy_df.append(pd.Series(['Linear-Logistic Regression', accuracy_text.format(train_acc1, test_acc1),
                                            accuracy_text.format(train_acc2, test_acc2),
                                            accuracy_text.format(train_acc3, test_acc3),],
                                           index=accuracy_df.columns), ignore_index=True)
accuracy_df

,Model,unigram-tfidf Accuracy,bigram-tfidf Accuracy,trigram-tfidf Accuracy
0,Naive Bayes,| Train : 51.323 ~~ Test: 51.292 |,| Train : 50.072 ~~ Test: 50.0 |,| Train : 49.846 ~~ Test: 49.815 |
1,Linear-Logistic Regression,| Train : 54.398 ~~ Test: 53.506 |,| Train : 52.204 ~~ Test: 51.292 |,| Train : 51.425 ~~ Test: 51.107 |


####**Support Vector Machines :**

In [0]:
# SVM on tfidf Vectors
train_acc1, test_acc1 = train_model(svm.SVC(), X_train_unigram_seq, y_train, X_test_unigram_seq, y_test)
train_acc2, test_acc2 = train_model(svm.SVC(), X_train_bigram_seq,  y_train, X_test_bigram_seq,  y_test)
train_acc3, test_acc3 = train_model(svm.SVC(), X_train_trigram_seq, y_train, X_test_trigram_seq, y_test)

accuracy_df = accuracy_df.append(pd.Series(['SVM Classifier', 
                                            accuracy_text.format(train_acc1, test_acc1),
                                            accuracy_text.format(train_acc2, test_acc2),
                                            accuracy_text.format(train_acc3, test_acc3),],
                                           index=accuracy_df.columns), ignore_index=True)
accuracy_df

,Model,unigram-tfidf Accuracy,bigram-tfidf Accuracy,trigram-tfidf Accuracy
0,Naive Bayes,| Train : 51.323 ~~ Test: 51.292 |,| Train : 50.072 ~~ Test: 50.0 |,| Train : 49.846 ~~ Test: 49.815 |
1,Linear-Logistic Regression,| Train : 54.398 ~~ Test: 53.506 |,| Train : 52.204 ~~ Test: 51.292 |,| Train : 51.425 ~~ Test: 51.107 |
2,SVM Classifier,| Train : 66.701 ~~ Test: 52.952 |,| Train : 65.737 ~~ Test: 51.292 |,| Train : 62.108 ~~ Test: 51.107 |


####**RandomForest Classifier :**

In [0]:
def randomforest_model(n_estimators,criterion,max_depth,min_samples_split):
  rf_model = ensemble.RandomForestClassifier(n_estimators=n_estimators, n_jobs=2, criterion = criterion,
                                          max_depth = max_depth, min_samples_split = min_samples_split, random_state=0)
  return rf_model

In [0]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(100, 1000, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [6, 10, 15]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split}

# Use the random grid to search for best hyperparameters
rf = ensemble.RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 50, cv = 3, verbose=2, random_state=42, n_jobs = 2)

In [0]:
# Fit the random search model
#rf_random.fit(X_train_trigram_seq, y_train)

In [0]:
#rf_random.best_params_

In [0]:
# RandomForest on tfidf Vectors
n_estimators = 600
criterion = 'gini'
max_depth = 100
min_samples_split = 6

unigram_rf_model = randomforest_model(n_estimators,criterion,100,6)
bigram_rf_model = randomforest_model(n_estimators,criterion,None,10)
trigram_rf_model = randomforest_model(n_estimators,criterion,None,10)

train_acc1, test_acc1 = train_model(unigram_rf_model, X_train_unigram_seq, y_train, X_test_unigram_seq, y_test)
train_acc2, test_acc2 = train_model(bigram_rf_model, X_train_bigram_seq,  y_train, X_test_bigram_seq,  y_test)
train_acc3, test_acc3 = train_model(trigram_rf_model, X_train_trigram_seq, y_train, X_test_trigram_seq, y_test)

accuracy_df = accuracy_df.append(pd.Series(['RandomForest: Esti = {},Cri={},max_depth={}'.format(n_estimators,criterion,max_depth), 
                                            accuracy_text.format(train_acc1, test_acc1),
                                            accuracy_text.format(train_acc2, test_acc2),
                                            accuracy_text.format(train_acc3, test_acc3),],
                                           index=accuracy_df.columns), ignore_index=True)

accuracy_df

,Model,unigram-tfidf Accuracy,bigram-tfidf Accuracy,trigram-tfidf Accuracy
0,Naive Bayes,| Train : 51.323 ~~ Test: 51.292 |,| Train : 50.072 ~~ Test: 50.0 |,| Train : 49.846 ~~ Test: 49.815 |
1,Linear-Logistic Regression,| Train : 54.398 ~~ Test: 53.506 |,| Train : 52.204 ~~ Test: 51.292 |,| Train : 51.425 ~~ Test: 51.107 |
2,SVM Classifier,| Train : 66.701 ~~ Test: 52.952 |,| Train : 65.737 ~~ Test: 51.292 |,| Train : 62.108 ~~ Test: 51.107 |
3,"RandomForest: Esti = 600,Cri=gini,max_depth=100",| Train : 63.728 ~~ Test: 54.982 |,| Train : 95.14 ~~ Test: 56.273 |,| Train : 96.76 ~~ Test: 55.535 |


####**XGBoost Classifier :**

In [0]:
# XGBoost on tfidf Vectors
train_acc1, test_acc1 = train_model(xgboost.XGBClassifier(), X_train_unigram_seq, y_train, X_test_unigram_seq, y_test)
train_acc2, test_acc2 = train_model(xgboost.XGBClassifier(), X_train_bigram_seq, y_train, X_test_bigram_seq, y_test)
train_acc3, test_acc3 = train_model(xgboost.XGBClassifier(), X_train_trigram_seq, y_train, X_test_trigram_seq, y_test)

In [0]:
accuracy_df = accuracy_df.append(pd.Series(['XGBoost', 
                                            accuracy_text.format(train_acc1, test_acc1),
                                            accuracy_text.format(train_acc2, test_acc2),
                                            accuracy_text.format(train_acc3, test_acc3),],
                                           index=accuracy_df.columns), ignore_index=True)
accuracy_df

,Model,unigram-tfidf Accuracy,bigram-tfidf Accuracy,trigram-tfidf Accuracy
0,Naive Bayes,| Train : 51.323 ~~ Test: 51.292 |,| Train : 50.072 ~~ Test: 50.0 |,| Train : 49.846 ~~ Test: 49.815 |
1,Linear-Logistic Regression,| Train : 54.398 ~~ Test: 53.506 |,| Train : 52.204 ~~ Test: 51.292 |,| Train : 51.425 ~~ Test: 51.107 |
2,SVM Classifier,| Train : 66.701 ~~ Test: 52.952 |,| Train : 65.737 ~~ Test: 51.292 |,| Train : 62.108 ~~ Test: 51.107 |
3,"RandomForest: Esti = 600,Cri=gini,max_depth=100",| Train : 63.728 ~~ Test: 54.982 |,| Train : 95.14 ~~ Test: 56.273 |,| Train : 96.76 ~~ Test: 55.535 |
4,XGBoost,| Train : 60.078 ~~ Test: 54.244 |,| Train : 60.139 ~~ Test: 53.506 |,| Train : 60.365 ~~ Test: 54.059 |


####**KNN Classifier:**

In [0]:
# KNN Classifier on tfidf Vectors
train_acc1, test_acc1 = train_model(neighbors.KNeighborsClassifier(n_neighbors=5, algorithm='brute'), X_train_unigram_seq, y_train, X_test_unigram_seq, y_test)
train_acc2, test_acc2 = train_model(neighbors.KNeighborsClassifier(n_neighbors=5, algorithm='brute'), X_train_bigram_seq, y_train, X_test_bigram_seq, y_test)
train_acc3, test_acc3 = train_model(neighbors.KNeighborsClassifier(n_neighbors=5, algorithm='brute'), X_train_trigram_seq, y_train, X_test_trigram_seq, y_test)

In [0]:
accuracy_df = accuracy_df.append(pd.Series(['KNN Classifier (k=5)', 
                                            accuracy_text.format(train_acc1, test_acc1),
                                            accuracy_text.format(train_acc2, test_acc2),
                                            accuracy_text.format(train_acc3, test_acc3),],
                                           index=accuracy_df.columns), ignore_index=True)
accuracy_df

,Model,unigram-tfidf Accuracy,bigram-tfidf Accuracy,trigram-tfidf Accuracy
0,Naive Bayes,| Train : 51.323 ~~ Test: 51.292 |,| Train : 50.072 ~~ Test: 50.0 |,| Train : 49.846 ~~ Test: 49.815 |
1,Linear-Logistic Regression,| Train : 54.398 ~~ Test: 53.506 |,| Train : 52.204 ~~ Test: 51.292 |,| Train : 51.425 ~~ Test: 51.107 |
2,SVM Classifier,| Train : 66.701 ~~ Test: 52.952 |,| Train : 65.737 ~~ Test: 51.292 |,| Train : 62.108 ~~ Test: 51.107 |
3,"RandomForest: Esti = 600,Cri=gini,max_depth=100",| Train : 63.728 ~~ Test: 54.982 |,| Train : 95.14 ~~ Test: 56.273 |,| Train : 96.76 ~~ Test: 55.535 |
4,XGBoost,| Train : 60.078 ~~ Test: 54.244 |,| Train : 60.139 ~~ Test: 53.506 |,| Train : 60.365 ~~ Test: 54.059 |
5,KNN Classifier (k=5),| Train : 55.198 ~~ Test: 52.214 |,| Train : 52.102 ~~ Test: 50.738 |,| Train : 51.097 ~~ Test: 50.185 |


##**Deep Learning Models :**

#####Pre-Requistic

In [27]:
token = Tokenizer()
#token.fit_on_texts(unigram_vocab['words'])
token.fit_on_texts(X_train)
word_index = token.word_index

X_train_seq = token.texts_to_sequences(X_train)
X_test_seq = token.texts_to_sequences(X_test)

vocab_size = len(word_index)
desc_max_length = cleansed_data_df['description_length'].max()
print('Vocabulary Size : ', vocab_size)
print('Max length of desc : ', desc_max_length)

# convert text to sequence of tokens and pad them to ensure equal length vectors 
X_train_seq_pad = pad_sequences(X_train_seq, maxlen=desc_max_length)
X_test_seq_pad = pad_sequences(X_test_seq, maxlen=desc_max_length)

Vocabulary Size :  3015
Max length of desc :  2900


In [28]:
print('1st row data : ', X_train.iloc[0])
print('1st row Seq : ', X_train_seq_pad[0])
print('100th row data : ', X_train.iloc[100])
print('100th row Seq : ', X_train_seq_pad[100])
print('1000th row data : ', X_train.iloc[1000])
print('1000th row Seq : ', X_train_seq_pad[1000])

1st row data :  patch installation patch installation
1st row Seq :  [   0    0    0 ...  223 1446  223]
100th row data :  production password reset account locked production password reset account locked
100th row Seq :  [ 0  0  0 ... 16 17 54]
1000th row data :  password change request password change request
1000th row Seq :  [ 0  0  0 ...  3 40 47]


In [0]:
y_train_cate = keras.utils.to_categorical(y_train)
y_test_cate = keras.utils.to_categorical(y_test)

In [30]:
print('Shape of y_train : ',y_train_cate.shape)
print('Shape of y_test : ',y_test_cate.shape)

Shape of y_train :  (4877, 52)
Shape of y_test :  (542, 52)


In [0]:
dl_accuracy_df = pd.DataFrame(columns = ['Model' , 'Accuracy']) 

In [0]:
accuracy_text = 'Test : {} '

In [0]:
stop = EarlyStopping(monitor="loss", patience=10, mode="min")
reduce_lr = ReduceLROnPlateau(monitor="loss", factor=0.1, patience=3, min_lr=1e-8, min_delta=0.0001, verbose=1, mode="min")
callback = [reduce_lr,stop]

In [0]:
epochs = 20
batch_size = 32
lr = 1e-2
Lambda = 1e-5

#####Embedding:

######Glove:

In [0]:
GLOVE_200d_EMBEDDING_FILE = PROJECT_DIR+"Data/glove.6B.200d.txt"

In [36]:
glove_embedding_index = {}
f = open(GLOVE_200d_EMBEDDING_FILE)
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    glove_embedding_index[word] = coefs
f.close()
len(glove_embedding_index)

400000

In [37]:
embed_dim=200
glove_embedding_matrix = np.zeros((vocab_size+1, embed_dim))
for word, i in token.word_index.items():
    glove_embedding_vector = glove_embedding_index.get(word)
    if glove_embedding_vector is not None:
        glove_embedding_matrix[i] = glove_embedding_vector
print('Shape of GloVe Embedding matrix : ', glove_embedding_matrix.shape)

Shape of GloVe Embedding matrix :  (3016, 200)


######StackOverflow:

In [0]:
SO_200d_EMBEDDING_FILE = PROJECT_DIR+"Data/SO_vectors_200.bin"

In [0]:
from gensim.models.keyedvectors import KeyedVectors as kv
so_word_vec = kv.load_word2vec_format(SO_200d_EMBEDDING_FILE, binary=True)

In [40]:
embed_dim=200
so_embedding_matrix = np.zeros((vocab_size+1, embed_dim))
for word, i in token.word_index.items():
  if word in so_word_vec.vocab:
    so_embedding_vector = so_word_vec[word]
    so_embedding_matrix[i] = so_embedding_vector
so_embedding_matrix.shape
print('Shape of GloVe Embedding matrix : ', so_embedding_matrix.shape)

Shape of GloVe Embedding matrix :  (3016, 200)


####Neural Networks :

#####Pre-requiste

In [0]:
def create_nn(input_size, output_size, embedding=False, vocab_size=None, embed_dim=None, embedding_matrix=None):
    model = Sequential()
    if(embedding):
      model.add(Embedding(vocab_size, embed_dim, input_length=input_size, weights=[embedding_matrix], trainable=False))
      #model.add(Embedding(vocab_size, embed_dim, input_length=input_size, ))
      model.add(Flatten())
      model.add(Dense(512,kernel_initializer='random_uniform', kernel_regularizer=regularizers.l2(Lambda)))
    else:
      model.add(Dense(512,input_dim=input_size, kernel_initializer='random_uniform', kernel_regularizer=regularizers.l2(Lambda)))
    
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.3))

    model.add(Dense(256,kernel_initializer='random_uniform',kernel_regularizer=regularizers.l2(Lambda)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.3))

    model.add(Dense(128,kernel_initializer='random_uniform',kernel_regularizer=regularizers.l2(Lambda)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.1))

    model.add(Dense(64, kernel_initializer='random_uniform',kernel_regularizer=regularizers.l2(Lambda)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    model.add(Dense(output_size, activation='softmax'))
    return model

In [0]:
def train_deep_model(model, feature_train, target_train, feature_test, target_test, epochs, batch_size, lr, Lambda, callbacks):
  model.compile(optimizer=optimizers.Adam(learning_rate=lr),
                loss='categorical_crossentropy', metrics=['accuracy'])
  history = model.fit(feature_train, target_train, validation_split=0.2, 
                      epochs=epochs, batch_size=batch_size, verbose=1,
                      callbacks = callbacks)
  eval_scores = model.evaluate(feature_test, target_test, verbose=0)
  return round((eval_scores[1]*100),2)

#####Vanilla NN:

In [227]:
vanilla_nn_model = create_nn(X_train_seq_pad.shape[1], y_train_cate.shape[1], 
                   False, vocab_size+1, embed_dim)
vanilla_nn_model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 512)               1485312   
_________________________________________________________________
batch_normalization_12 (Batc (None, 512)               2048      
_________________________________________________________________
activation_12 (Activation)   (None, 512)               0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 256)               131328    
_________________________________________________________________
batch_normalization_13 (Batc (None, 256)               1024      
_________________________________________________________________
activation_13 (Activation)   (None, 256)              

In [228]:
test_acc = train_deep_model(vanilla_nn_model, X_train_seq_pad, y_train_cate, X_test_seq_pad, y_test_cate, 
                             epochs, batch_size, lr, Lambda, callback)

Epoch 1/50
31/31 [==============================] - 1s 27ms/step - loss: 2.8376 - accuracy: 0.4304 - val_loss: 9.2343 - val_accuracy: 0.2756 - lr: 0.0100
Epoch 2/50
31/31 [==============================] - 1s 21ms/step - loss: 2.4912 - accuracy: 0.4796 - val_loss: 3.6304 - val_accuracy: 0.4006 - lr: 0.0100
Epoch 3/50
31/31 [==============================] - 1s 21ms/step - loss: 2.4511 - accuracy: 0.4853 - val_loss: 2.9140 - val_accuracy: 0.4867 - lr: 0.0100
Epoch 4/50
31/31 [==============================] - 1s 21ms/step - loss: 2.4401 - accuracy: 0.4883 - val_loss: 2.6365 - val_accuracy: 0.4764 - lr: 0.0100
Epoch 5/50
31/31 [==============================] - 1s 21ms/step - loss: 2.4110 - accuracy: 0.4917 - val_loss: 2.5712 - val_accuracy: 0.4887 - lr: 0.0100
Epoch 6/50
31/31 [==============================] - 1s 21ms/step - loss: 2.3838 - accuracy: 0.4901 - val_loss: 2.5308 - val_accuracy: 0.4898 - lr: 0.0100
Epoch 7/50
31/31 [==============================] - 1s 21ms/step - loss: 2.3

In [230]:
dl_accuracy_df = dl_accuracy_df.append(pd.Series(['Vanilla NN', accuracy_text.format(test_acc)],
                                           index=dl_accuracy_df.columns), ignore_index=True)
dl_accuracy_df

,Model,Accuracy
0,Simple NN,Test : 45.76
1,Vanilla NN,Test : 45.76


In [0]:
vanilla_nn_model.save_weights(PROJECT_DIR+"weights/vanilla_nn_weights.h5")
#vanilla_nn_model.load_weights(PROJECT_DIR+"weights/vanilla_nn_weights.h5")

#####**NN Model with Pretrained Embedding with GloVe :**

In [242]:
nn_glove200_model = create_nn(X_train_seq_pad.shape[1], y_train_cate.shape[1], 
                   True, vocab_size+1, embed_dim, glove_embedding_matrix)

nn_glove200_model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 2900, 200)         589400    
_________________________________________________________________
flatten_1 (Flatten)          (None, 580000)            0         
_________________________________________________________________
dense_25 (Dense)             (None, 512)               296960512 
_________________________________________________________________
batch_normalization_20 (Batc (None, 512)               2048      
_________________________________________________________________
activation_20 (Activation)   (None, 512)               0         
_________________________________________________________________
dropout_15 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_26 (Dense)             (None, 256)             

In [243]:
test_acc = train_deep_model(nn_glove200_model, X_train_seq_pad, y_train_cate, X_test_seq_pad, y_test_cate, 
                             epochs, batch_size, lr, Lambda, callback)

Epoch 1/20
31/31 [==============================] - 62s 2s/step - loss: 3.4616 - accuracy: 0.4381 - val_loss: 6.2195 - val_accuracy: 0.4314 - lr: 0.0100
Epoch 2/20
31/31 [==============================] - 62s 2s/step - loss: 2.6399 - accuracy: 0.4912 - val_loss: 3.0806 - val_accuracy: 0.4877 - lr: 0.0100
Epoch 3/20
31/31 [==============================] - 62s 2s/step - loss: 2.4571 - accuracy: 0.4978 - val_loss: 2.8552 - val_accuracy: 0.4887 - lr: 0.0100
Epoch 4/20
31/31 [==============================] - 62s 2s/step - loss: 2.3515 - accuracy: 0.5096 - val_loss: 2.7122 - val_accuracy: 0.5061 - lr: 0.0100
Epoch 5/20
31/31 [==============================] - 63s 2s/step - loss: 2.2605 - accuracy: 0.5347 - val_loss: 2.8648 - val_accuracy: 0.5041 - lr: 0.0100
Epoch 6/20
31/31 [==============================] - 62s 2s/step - loss: 2.2051 - accuracy: 0.5588 - val_loss: 2.9137 - val_accuracy: 0.4980 - lr: 0.0100
Epoch 7/20
31/31 [==============================] - 62s 2s/step - loss: 2.0672 - a

In [244]:
dl_accuracy_df = dl_accuracy_df.append(pd.Series(['NN-glove200-trainable-true-initiallearning', 
                                            accuracy_text.format(test_acc)],
                                           index=dl_accuracy_df.columns), ignore_index=True)
dl_accuracy_df

,Model,Accuracy
0,Simple NN,Test : 45.76
1,Vanilla NN,Test : 45.76
2,NN-glove200-trainable-true-initiallearning,Test : 45.76
3,NN-glove200-trainable-true-initiallearning,Test : 50.92


In [0]:
nn_glove200_model.save_weights(PROJECT_DIR+"weights/nn_glove200_weights.h5")
#nn_own_embed_model.load_weights(PROJECT_DIR+"weights/nn_own_embed_weights.h5")

In [0]:
for layer in nn_glove200_model.layers:
  if 'embedding' in (layer.name.lower()):
    layer.trainable = True
    break

In [247]:
nn_glove200_model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 2900, 200)         589400    
_________________________________________________________________
flatten_1 (Flatten)          (None, 580000)            0         
_________________________________________________________________
dense_25 (Dense)             (None, 512)               296960512 
_________________________________________________________________
batch_normalization_20 (Batc (None, 512)               2048      
_________________________________________________________________
activation_20 (Activation)   (None, 512)               0         
_________________________________________________________________
dropout_15 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_26 (Dense)             (None, 256)             

In [248]:
test_acc = train_deep_model(nn_glove200_model, X_train_seq_pad, y_train_cate, X_test_seq_pad, y_test_cate, 
                                                epochs, batch_size, lr, Lambda, callback)

Epoch 1/20
31/31 [==============================] - 87s 3s/step - loss: 1.5537 - accuracy: 0.8129 - val_loss: 4.4915 - val_accuracy: 0.0502 - lr: 0.0100
Epoch 2/20
31/31 [==============================] - 86s 3s/step - loss: 1.6009 - accuracy: 0.8239 - val_loss: 3.9679 - val_accuracy: 0.4447 - lr: 0.0100
Epoch 3/20
31/31 [==============================] - 85s 3s/step - loss: 1.5027 - accuracy: 0.8703 - val_loss: 4.7072 - val_accuracy: 0.5072 - lr: 0.0100
Epoch 4/20
31/31 [==============================] - 84s 3s/step - loss: 1.3971 - accuracy: 0.8918 - val_loss: 4.7462 - val_accuracy: 0.4652 - lr: 0.0100
Epoch 5/20
31/31 [==============================] - 89s 3s/step - loss: 1.2488 - accuracy: 0.9100 - val_loss: 5.4832 - val_accuracy: 0.5051 - lr: 0.0100
Epoch 6/20
31/31 [==============================] - 84s 3s/step - loss: 1.2470 - accuracy: 0.9154 - val_loss: 4.7271 - val_accuracy: 0.5041 - lr: 0.0100
Epoch 7/20
31/31 [==============================] - 85s 3s/step - loss: 1.1978 - a

In [249]:
dl_accuracy_df = dl_accuracy_df.append(pd.Series(['NN-glove200-trainable-true-finetuning',
                                            accuracy_text.format(test_acc)],
                                           index=dl_accuracy_df.columns), ignore_index=True)
dl_accuracy_df

,Model,Accuracy
0,Simple NN,Test : 45.76
1,Vanilla NN,Test : 45.76
2,NN-glove200-trainable-true-initiallearning,Test : 45.76
3,NN-glove200-trainable-true-initiallearning,Test : 50.92
4,NN-glove200-trainable-true-finetuning,Test : 51.29


In [0]:
nn_glove200_model.save_weights(PROJECT_DIR+"weights/nn_glove200_finetuned_weights.h5")
#nn_own_embed_model.load_weights(PROJECT_DIR+"weights/nn_own_embed_weights.h5")

#####**NN Model with Pretrained Embedding with StackOverflow :**

In [252]:
nn_stackoverflow_200_model = create_nn(X_train_seq_pad.shape[1], y_train_cate.shape[1], 
                   True, vocab_size+1, embed_dim, so_embedding_matrix)

nn_stackoverflow_200_model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 2900, 200)         589400    
_________________________________________________________________
flatten_2 (Flatten)          (None, 580000)            0         
_________________________________________________________________
dense_30 (Dense)             (None, 512)               296960512 
_________________________________________________________________
batch_normalization_24 (Batc (None, 512)               2048      
_________________________________________________________________
activation_24 (Activation)   (None, 512)               0         
_________________________________________________________________
dropout_18 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_31 (Dense)             (None, 256)             

In [253]:
test_acc = train_deep_model(nn_stackoverflow_200_model, X_train_seq_pad, y_train_cate, X_test_seq_pad, y_test_cate, 
                             epochs, batch_size, lr, Lambda, callback)


Epoch 1/20
31/31 [==============================] - 63s 2s/step - loss: 3.4356 - accuracy: 0.4425 - val_loss: 5.5999 - val_accuracy: 0.4887 - lr: 0.0100
Epoch 2/20
31/31 [==============================] - 62s 2s/step - loss: 2.6567 - accuracy: 0.4994 - val_loss: 3.0151 - val_accuracy: 0.4652 - lr: 0.0100
Epoch 3/20
31/31 [==============================] - 62s 2s/step - loss: 2.3859 - accuracy: 0.5265 - val_loss: 2.8248 - val_accuracy: 0.4744 - lr: 0.0100
Epoch 4/20
31/31 [==============================] - 62s 2s/step - loss: 2.2699 - accuracy: 0.5460 - val_loss: 2.8156 - val_accuracy: 0.5000 - lr: 0.0100
Epoch 5/20
31/31 [==============================] - 62s 2s/step - loss: 2.2629 - accuracy: 0.5688 - val_loss: 3.0139 - val_accuracy: 0.4990 - lr: 0.0100
Epoch 6/20
31/31 [==============================] - 65s 2s/step - loss: 2.0743 - accuracy: 0.6011 - val_loss: 2.9565 - val_accuracy: 0.5154 - lr: 0.0100
Epoch 7/20
31/31 [==============================] - 62s 2s/step - loss: 1.9164 - a

In [254]:
dl_accuracy_df = dl_accuracy_df.append(pd.Series(['NN-so200-trainable-initiallearning', 
                                            accuracy_text.format(test_acc)],
                                           index=dl_accuracy_df.columns), ignore_index=True)
dl_accuracy_df

,Model,Accuracy
0,Simple NN,Test : 45.76
1,Vanilla NN,Test : 45.76
2,NN-glove200-trainable-true-initiallearning,Test : 45.76
3,NN-glove200-trainable-true-initiallearning,Test : 50.92
4,NN-glove200-trainable-true-finetuning,Test : 51.29
5,NN-so200-trainable-initiallearning,Test : 50.92


In [0]:
nn_stackoverflow_200_model.save_weights(PROJECT_DIR+"weights/nn_so200_weights.h5")

In [256]:
for layer in nn_stackoverflow_200_model.layers:
  if 'embedding' in (layer.name.lower()):
    layer.trainable = True
    break

nn_stackoverflow_200_model.summary()


Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 2900, 200)         589400    
_________________________________________________________________
flatten_2 (Flatten)          (None, 580000)            0         
_________________________________________________________________
dense_30 (Dense)             (None, 512)               296960512 
_________________________________________________________________
batch_normalization_24 (Batc (None, 512)               2048      
_________________________________________________________________
activation_24 (Activation)   (None, 512)               0         
_________________________________________________________________
dropout_18 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_31 (Dense)             (None, 256)             

In [257]:
test_acc = train_deep_model(nn_stackoverflow_200_model, X_train_seq_pad, y_train_cate, X_test_seq_pad, y_test_cate, 
                                                epochs, batch_size, lr, Lambda, callback)


Epoch 1/20
31/31 [==============================] - 88s 3s/step - loss: 1.3083 - accuracy: 0.8185 - val_loss: 5.1011 - val_accuracy: 0.5010 - lr: 0.0100
Epoch 2/20
31/31 [==============================] - 86s 3s/step - loss: 1.7555 - accuracy: 0.7829 - val_loss: 4.8135 - val_accuracy: 0.2777 - lr: 0.0100
Epoch 3/20
31/31 [==============================] - 86s 3s/step - loss: 1.6176 - accuracy: 0.8285 - val_loss: 5.1695 - val_accuracy: 0.4590 - lr: 0.0100
Epoch 4/20
31/31 [==============================] - ETA: 0s - loss: 1.4357 - accuracy: 0.8641
Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.0009999999776482583.
31/31 [==============================] - 86s 3s/step - loss: 1.4357 - accuracy: 0.8641 - val_loss: 4.9326 - val_accuracy: 0.5092 - lr: 0.0100
Epoch 5/20
31/31 [==============================] - 86s 3s/step - loss: 1.2815 - accuracy: 0.8959 - val_loss: 4.9363 - val_accuracy: 0.5092 - lr: 1.0000e-03
Epoch 6/20
31/31 [==============================] - 86s 3s/step - lo

In [258]:
dl_accuracy_df = dl_accuracy_df.append(pd.Series(['NN-so200-trainable-true-finetuning',
                                            accuracy_text.format(test_acc)],
                                           index=dl_accuracy_df.columns), ignore_index=True)
dl_accuracy_df


,Model,Accuracy
0,Simple NN,Test : 45.76
1,Vanilla NN,Test : 45.76
2,NN-glove200-trainable-true-initiallearning,Test : 45.76
3,NN-glove200-trainable-true-initiallearning,Test : 50.92
4,NN-glove200-trainable-true-finetuning,Test : 51.29
5,NN-so200-trainable-initiallearning,Test : 50.92
6,NN-so200-trainable-true-finetuning,Test : 48.34


In [0]:
nn_stackoverflow_200_model.save_weights(PROJECT_DIR+"weights/nn_so200_finetuned_weights.h5")


###Convolutional Neural Network

In [0]:
def create_cnn(input_size, output_size, embedding=False, vocab_size=None, embed_dim=None, embedding_matrix=None):
    model = Sequential()
    model.add(Embedding(vocab_size, embed_dim, input_length=input_size, weights=[embedding_matrix], trainable=False))
    #1st hidden layer
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    #2nd hidden layer
    model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    #3rd hidden layer
    model.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    
    model.add(Flatten())
    model.add(Dense(64, kernel_initializer='random_uniform',kernel_regularizer=regularizers.l2(Lambda)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    model.add(Dense(output_size, activation='softmax'))
    return model

####Glove:

In [53]:
cnn_glove_200_model = create_cnn(X_train_seq_pad.shape[1], y_train_cate.shape[1], 
                   True, vocab_size+1, embed_dim, glove_embedding_matrix)

cnn_glove_200_model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 2900, 200)         589400    
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 2898, 64)          38464     
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 1449, 64)          0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 1447, 128)         24704     
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 723, 128)          0         
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 721, 256)          98560     
_________________________________________________________________
max_pooling1d_8 (MaxPooling1 (None, 360, 256)         

In [54]:
test_acc = train_deep_model(cnn_glove_200_model, X_train_seq_pad, y_train_cate, X_test_seq_pad, y_test_cate, 
                             epochs, batch_size, lr, Lambda, callback)


Epoch 1/50
31/31 [==============================] - 53s 2s/step - loss: 2.7697 - accuracy: 0.4524 - val_loss: 3.7932 - val_accuracy: 0.1352 - lr: 0.0100
Epoch 2/50
31/31 [==============================] - 52s 2s/step - loss: 2.4357 - accuracy: 0.4896 - val_loss: 3.2143 - val_accuracy: 0.4969 - lr: 0.0100
Epoch 3/50
31/31 [==============================] - 52s 2s/step - loss: 2.3899 - accuracy: 0.4896 - val_loss: 2.6848 - val_accuracy: 0.4990 - lr: 0.0100
Epoch 4/50
31/31 [==============================] - 52s 2s/step - loss: 2.3473 - accuracy: 0.4917 - val_loss: 2.5896 - val_accuracy: 0.5010 - lr: 0.0100
Epoch 5/50
31/31 [==============================] - 52s 2s/step - loss: 2.3272 - accuracy: 0.4935 - val_loss: 2.6168 - val_accuracy: 0.4990 - lr: 0.0100
Epoch 6/50
31/31 [==============================] - 52s 2s/step - loss: 2.2885 - accuracy: 0.4996 - val_loss: 2.7182 - val_accuracy: 0.4959 - lr: 0.0100
Epoch 7/50
31/31 [==============================] - 52s 2s/step - loss: 2.2249 - a

In [59]:
dl_accuracy_df = dl_accuracy_df.append(pd.Series(['CNN-Glove200-trainable-initiallearning', 
                                            accuracy_text.format(test_acc)],
                                           index=dl_accuracy_df.columns), ignore_index=True)
dl_accuracy_df

,Model,Accuracy
0,CNN-Glove200-trainable-initiallearning,Test : 46.68


In [0]:
cnn_glove_200_model.save_weights(PROJECT_DIR+"weights/cnn_glove200_weights.h5")

In [61]:
for layer in cnn_glove_200_model.layers:
  if 'embedding' in (layer.name.lower()):
    layer.trainable = True
    break

cnn_glove_200_model.summary()


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 2900, 200)         589400    
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 2898, 64)          38464     
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 1449, 64)          0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 1447, 128)         24704     
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 723, 128)          0         
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 721, 256)          98560     
_________________________________________________________________
max_pooling1d_8 (MaxPooling1 (None, 360, 256)         

In [62]:
test_acc = train_deep_model(cnn_glove_200_model, X_train_seq_pad, y_train_cate, X_test_seq_pad, y_test_cate, 
                                                epochs, batch_size, lr, Lambda, callback)


Epoch 1/50
31/31 [==============================] - 60s 2s/step - loss: 1.3129 - accuracy: 0.7296 - val_loss: 13.0563 - val_accuracy: 0.5020 - lr: 0.0100
Epoch 2/50
31/31 [==============================] - 60s 2s/step - loss: 1.4590 - accuracy: 0.6944 - val_loss: 12.5960 - val_accuracy: 0.4805 - lr: 0.0100
Epoch 3/50
31/31 [==============================] - 59s 2s/step - loss: 1.3235 - accuracy: 0.7267 - val_loss: 5.7537 - val_accuracy: 0.4662 - lr: 0.0100
Epoch 4/50
31/31 [==============================] - 62s 2s/step - loss: 1.2599 - accuracy: 0.7431 - val_loss: 11.3370 - val_accuracy: 0.1383 - lr: 0.0100
Epoch 5/50
31/31 [==============================] - 60s 2s/step - loss: 1.2222 - accuracy: 0.7524 - val_loss: 7.6528 - val_accuracy: 0.1537 - lr: 0.0100
Epoch 6/50
31/31 [==============================] - 60s 2s/step - loss: 1.2026 - accuracy: 0.7613 - val_loss: 5.0474 - val_accuracy: 0.1475 - lr: 0.0100
Epoch 7/50
31/31 [==============================] - 60s 2s/step - loss: 1.1849 

In [63]:
dl_accuracy_df = dl_accuracy_df.append(pd.Series(['CNN-Glove200-trainable-true-finetuning',
                                            accuracy_text.format(test_acc)],
                                           index=dl_accuracy_df.columns), ignore_index=True)
dl_accuracy_df


,Model,Accuracy
0,CNN-Glove200-trainable-initiallearning,Test : 46.68
1,CNN-Glove200-trainable-true-finetuning,Test : 46.31


In [0]:
cnn_glove_200_model.save_weights(PROJECT_DIR+"weights/cnn_glove200_finetuned_weights.h5")

####Stackoverflow:

In [65]:
cnn_stackoverflow_200_model = create_cnn(X_train_seq_pad.shape[1], y_train_cate.shape[1], 
                   True, vocab_size+1, embed_dim, so_embedding_matrix)

cnn_stackoverflow_200_model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 2900, 200)         589400    
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 2898, 64)          38464     
_________________________________________________________________
max_pooling1d_9 (MaxPooling1 (None, 1449, 64)          0         
_________________________________________________________________
conv1d_11 (Conv1D)           (None, 1447, 128)         24704     
_________________________________________________________________
max_pooling1d_10 (MaxPooling (None, 723, 128)          0         
_________________________________________________________________
conv1d_12 (Conv1D)           (None, 721, 256)          98560     
_________________________________________________________________
max_pooling1d_11 (MaxPooling (None, 360, 256)         

In [66]:
test_acc = train_deep_model(cnn_stackoverflow_200_model, X_train_seq_pad, y_train_cate, X_test_seq_pad, y_test_cate, 
                             epochs, batch_size, lr, Lambda, callback)


Epoch 1/50
31/31 [==============================] - 52s 2s/step - loss: 2.8170 - accuracy: 0.4366 - val_loss: 3.3295 - val_accuracy: 0.4918 - lr: 0.0100
Epoch 2/50
31/31 [==============================] - 51s 2s/step - loss: 2.4653 - accuracy: 0.4883 - val_loss: 2.8042 - val_accuracy: 0.5010 - lr: 0.0100
Epoch 3/50
31/31 [==============================] - 51s 2s/step - loss: 2.4079 - accuracy: 0.4901 - val_loss: 2.6365 - val_accuracy: 0.5000 - lr: 0.0100
Epoch 4/50
31/31 [==============================] - 52s 2s/step - loss: 2.3765 - accuracy: 0.4930 - val_loss: 2.5612 - val_accuracy: 0.5010 - lr: 0.0100
Epoch 5/50
31/31 [==============================] - 52s 2s/step - loss: 2.3320 - accuracy: 0.4940 - val_loss: 2.4853 - val_accuracy: 0.5010 - lr: 0.0100
Epoch 6/50
31/31 [==============================] - 52s 2s/step - loss: 2.2978 - accuracy: 0.4950 - val_loss: 2.5010 - val_accuracy: 0.4969 - lr: 0.0100
Epoch 7/50
31/31 [==============================] - 52s 2s/step - loss: 2.2614 - a

In [67]:
dl_accuracy_df = dl_accuracy_df.append(pd.Series(['CNN-SO200-trainable-initiallearning', 
                                            accuracy_text.format(test_acc)],
                                           index=dl_accuracy_df.columns), ignore_index=True)
dl_accuracy_df

,Model,Accuracy
0,CNN-Glove200-trainable-initiallearning,Test : 46.68
1,CNN-Glove200-trainable-true-finetuning,Test : 46.31
2,CNN-SO200-trainable-initiallearning,Test : 45.57


In [0]:
cnn_stackoverflow_200_model.save_weights(PROJECT_DIR+"weights/cnn_so200_weights.h5")

In [69]:
for layer in cnn_stackoverflow_200_model.layers:
  if 'embedding' in (layer.name.lower()):
    layer.trainable = True
    break

cnn_stackoverflow_200_model.summary()


Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 2900, 200)         589400    
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 2898, 64)          38464     
_________________________________________________________________
max_pooling1d_9 (MaxPooling1 (None, 1449, 64)          0         
_________________________________________________________________
conv1d_11 (Conv1D)           (None, 1447, 128)         24704     
_________________________________________________________________
max_pooling1d_10 (MaxPooling (None, 723, 128)          0         
_________________________________________________________________
conv1d_12 (Conv1D)           (None, 721, 256)          98560     
_________________________________________________________________
max_pooling1d_11 (MaxPooling (None, 360, 256)         

In [70]:
test_acc = train_deep_model(cnn_stackoverflow_200_model, X_train_seq_pad, y_train_cate, X_test_seq_pad, y_test_cate, 
                                                epochs, batch_size, lr, Lambda, callback)


Epoch 1/50
31/31 [==============================] - 60s 2s/step - loss: 1.2868 - accuracy: 0.7406 - val_loss: 8.2965 - val_accuracy: 0.4375 - lr: 0.0100
Epoch 2/50
31/31 [==============================] - 60s 2s/step - loss: 1.2852 - accuracy: 0.7319 - val_loss: 11.0030 - val_accuracy: 0.1465 - lr: 0.0100
Epoch 3/50
31/31 [==============================] - 59s 2s/step - loss: 1.2476 - accuracy: 0.7467 - val_loss: 8.9344 - val_accuracy: 0.1199 - lr: 0.0100
Epoch 4/50
31/31 [==============================] - 59s 2s/step - loss: 1.2381 - accuracy: 0.7496 - val_loss: 11.1649 - val_accuracy: 0.4857 - lr: 0.0100
Epoch 5/50
31/31 [==============================] - 62s 2s/step - loss: 1.2639 - accuracy: 0.7439 - val_loss: 7.6527 - val_accuracy: 0.4488 - lr: 0.0100
Epoch 6/50
31/31 [==============================] - 60s 2s/step - loss: 1.2246 - accuracy: 0.7537 - val_loss: 5.2341 - val_accuracy: 0.4436 - lr: 0.0100
Epoch 7/50
31/31 [==============================] - 60s 2s/step - loss: 1.2035 -

In [71]:
dl_accuracy_df = dl_accuracy_df.append(pd.Series(['CNN-SO200-trainable-true-finetuning',
                                            accuracy_text.format(test_acc)],
                                           index=dl_accuracy_df.columns), ignore_index=True)
dl_accuracy_df


,Model,Accuracy
0,CNN-Glove200-trainable-initiallearning,Test : 46.68
1,CNN-Glove200-trainable-true-finetuning,Test : 46.31
2,CNN-SO200-trainable-initiallearning,Test : 45.57
3,CNN-SO200-trainable-true-finetuning,Test : 47.05


In [0]:
cnn_stackoverflow_200_model.save_weights(PROJECT_DIR+"weights/cnn_so200_finetuned_weights.h5")

####CNN - MultiChannel

In [0]:
def create_multi_channel_cnn(input_size, output_size, embedding=False, vocab_size=None, embed_dim=None, embedding_matrix=None):

    #channel 1
    inputs1 = Input(shape=(input_size,))
    embedding1 = Embedding(vocab_size, embed_dim, weights=[embedding_matrix], trainable=False)(inputs1)
    conv1_1 = Conv1D(filters=64, kernel_size=2, activation='relu')(embedding1)
    pool1_1 = MaxPooling1D(pool_size=2)(conv1_1)
    conv1_2 = Conv1D(filters=128, kernel_size=4, activation='relu')(pool1_1)
    pool1_2 = MaxPooling1D(pool_size=2)(conv1_2)
    conv1_3 = Conv1D(filters=256, kernel_size=4, activation='relu')(pool1_2)
    pool1_3 = MaxPooling1D(pool_size=2)(conv1_3)
    flat1 = Flatten()(pool1_3)

    #channel 2
    inputs2 = Input(shape=(input_size,))
    embedding2 = Embedding(vocab_size, embed_dim, weights=[embedding_matrix], trainable=False)(inputs2)
    conv2_1 = Conv1D(filters=64, kernel_size=3, activation='relu')(embedding2)
    pool2_1 = MaxPooling1D(pool_size=2)(conv2_1)
    conv2_2 = Conv1D(filters=128, kernel_size=3, activation='relu')(pool2_1)
    pool2_2 = MaxPooling1D(pool_size=2)(conv2_2)
    conv2_3 = Conv1D(filters=256, kernel_size=3, activation='relu')(pool2_2)
    pool2_3 = MaxPooling1D(pool_size=2)(conv2_3)
    flat2 = Flatten()(pool2_3)

    #channel 3
    inputs3 = Input(shape=(input_size,))
    embedding3 = Embedding(vocab_size, embed_dim, weights=[embedding_matrix], trainable=False)(inputs3)
    conv3_1 = Conv1D(filters=64, kernel_size=4, activation='relu')(embedding3)
    pool3_1 = MaxPooling1D(pool_size=2)(conv3_1)
    conv3_2 = Conv1D(filters=64, kernel_size=4, activation='relu')(pool3_1)
    pool3_2 = MaxPooling1D(pool_size=2)(conv3_2)
    conv3_3 = Conv1D(filters=64, kernel_size=4, activation='relu')(pool3_2)
    pool3_3 = MaxPooling1D(pool_size=2)(conv3_3)
    flat3 = Flatten()(pool3_3)

    #join input channels
    channel_layers = Concatenate([flat1, flat2, flat3])

    mid_dense1 = Dense(64, kernel_initializer='random_uniform',kernel_regularizer=regularizers.l2(Lambda))(channel_layers)
    mid_norm1 = BatchNormalization()(mid_dense1)
    mid_actv1 = Activation('relu')(mid_norm1)

    output = Dense(output_size, activation='softmax')(mid_actv1)
    model = models.Model(inputs=[inputs1, inputs2, inputs3], outputs=output)
    return model

In [0]:
def train_deep_multichannel_model(model, feature_train, target_train, feature_test, target_test, epochs, batch_size, lr, Lambda, callbacks):
  model.compile(optimizer=optimizers.Adam(learning_rate=lr),
                loss='categorical_crossentropy', metrics=['accuracy'])
  history = model.fit([feature_train,feature_train,feature_train], target_train, validation_split=0.2, 
                      epochs=epochs, batch_size=batch_size, verbose=1,
                      callbacks = callbacks)
  #predict = model.predict_classes(feature_validate)
  eval_scores = model.evaluate([feature_test,feature_test,feature_test], target_test, verbose=0)
  return round((eval_scores[1]*100),2)

#####Glove

In [91]:
cnn_multi_channel_glove200_model = create_multi_channel_cnn(X_train_seq_pad.shape[1], y_train_cate.shape[1], 
                   True, vocab_size+1, embed_dim, glove_embedding_matrix)

cnn_multi_channel_glove200_model.summary()

AttributeError: ignored

In [0]:
plot_model(cnn_multi_channel_glove200_model, show_shapes=True, to_file='multichannel.png')

In [0]:
test_acc = train_deep_multichannel_model(cnn_multi_channel_glove200_model, X_train_seq_pad, y_train_cate, X_test_seq_pad, y_test_cate, 
                             epochs, batch_size, lr, Lambda, callback)

In [0]:

dl_accuracy_df = dl_accuracy_df.append(pd.Series(['CNN-MultiChannel-glove200-trainable-initiallearning', 
                                            accuracy_text.format(test_acc)],
                                           index=dl_accuracy_df.columns), ignore_index=True)
dl_accuracy_df

In [0]:
cnn_multi_channel_glove200_model.save_weights(PROJECT_DIR+"weights/cnn_glove200_weights.h5")

In [0]:
for layer in cnn_multi_channel_glove200_model.layers:
  if 'embedding' in (layer.name.lower()):
    layer.trainable = True
    break


In [0]:
test_acc = train_deep_multichannel_model(cnn_multi_channel_glove200_model, X_train_seq_pad, y_train_cate, X_test_seq_pad, y_test_cate, 
                                                epochs, batch_size, lr, Lambda, callback)


In [0]:
dl_accuracy_df = dl_accuracy_df.append(pd.Series(['CNN-MultiChannel-glove200-trainable-true-finetuning',
                                            accuracy_text.format(test_acc)],
                                           index=dl_accuracy_df.columns), ignore_index=True)
dl_accuracy_df

In [0]:
cnn_multi_channel_so200_model.save_weights(PROJECT_DIR+"weights/cnn_multichannel_glove200_finetuned_weights.h5")

#####Stackoverflow

In [0]:
cnn_multi_channel_so200_model = create_multi_channel_cnn(X_train_seq_pad.shape[1], y_train_cate.shape[1], 
                   True, vocab_size+1, embed_dim, so_embedding_matrix)

cnn_multi_channel_so200_model.summary()

In [0]:
plot_model(cnn_multi_channel_so200_model, show_shapes=True, to_file='multichannel.png')

In [0]:
test_acc = train_deep_multichannel_model(cnn_multi_channel_so200_model, X_train_seq_pad, y_train_cate, X_test_seq_pad, y_test_cate, 
                             epochs, batch_size, lr, Lambda, callback)

In [0]:
dl_accuracy_df = dl_accuracy_df.append(pd.Series(['CNN-MultiChannel-SO200-trainable-initiallearning', 
                                            accuracy_text.format(test_acc)],
                                           index=dl_accuracy_df.columns), ignore_index=True)
dl_accuracy_df

In [0]:
cnn_multi_channel_so200_model.save_weights(PROJECT_DIR+"weights/cnn_so200_weights.h5")

In [0]:
for layer in cnn_multi_channel_so200_model.layers:
  if 'embedding' in (layer.name.lower()):
    layer.trainable = True
    break

In [0]:
test_acc = train_deep_multichannel_model(cnn_multi_channel_so200_model, X_train_seq_pad, y_train_cate, X_test_seq_pad, y_test_cate, 
                                                epochs, batch_size, lr, Lambda, callback)


In [0]:
dl_accuracy_df = dl_accuracy_df.append(pd.Series(['CNN-MultiChannel-SO200-trainable-true-finetuning',
                                            accuracy_text.format(test_acc)],
                                           index=dl_accuracy_df.columns), ignore_index=True)
dl_accuracy_df


In [0]:
cnn_multi_channel_so200_model.save_weights(PROJECT_DIR+"weights/cnn_multichannel_so200_finetuned_weights.h5")

###Recurrent Neural Network

####RNN - Vanilla

In [0]:
def create_rnn(input_size, output_size, embedding=False, vocab_size=None, embed_dim=None, embedding_matrix=None):
    model = Sequential()
    
    model.add(Embedding(vocab_size, embed_dim, input_length=input_size, weights=[embedding_matrix], trainable=False))
    
    model.add(SimpleRNN(200,return_sequences=True))
    model.add(SimpleRNN(100,return_sequences=False))

    model.add(Dense(64, kernel_initializer='random_uniform',kernel_regularizer=regularizers.l2(Lambda)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    model.add(Dense(output_size, activation='softmax'))
    return model

#####Glove:

In [47]:
rnn_glove_200_model = create_rnn(X_train_seq_pad.shape[1], y_train_cate.shape[1], 
                   True, vocab_size+1, embed_dim, glove_embedding_matrix)

rnn_glove_200_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 2900, 200)         603200    
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 2900, 200)         80200     
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 100)               30100     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                6464      
_________________________________________________________________
batch_normalization_1 (Batch (None, 64)                256       
_________________________________________________________________
activation_1 (Activation)    (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 52)               

In [0]:
test_acc = train_deep_model(rnn_glove_200_model, X_train_seq_pad, y_train_cate, X_test_seq_pad, y_test_cate, 
                             epochs, batch_size, lr, Lambda, callback)


Epoch 1/20
 69/122 [===============>..............] - ETA: 2:18 - loss: 3.0687 - accuracy: 0.4144

In [104]:
dl_accuracy_df = dl_accuracy_df.append(pd.Series(['RNN-Glove-200-trainable-initiallearning', 
                                            accuracy_text.format(test_acc)],
                                           index=dl_accuracy_df.columns), ignore_index=True)
dl_accuracy_df

,Model,Accuracy
0,CNN-Glove200-trainable-initiallearning,Test : 46.68
1,CNN-Glove200-trainable-true-finetuning,Test : 46.31
2,CNN-SO200-trainable-initiallearning,Test : 45.57
3,CNN-SO200-trainable-true-finetuning,Test : 47.05
4,RNN-Glove-200-trainable-initiallearning,Test : 48.34


In [0]:
rnn_glove_200_model.save_weights(PROJECT_DIR+"weights/rnn_glove_200_weights.h5")

In [0]:
for layer in rnn_glove_200_model.layers:
  if 'embedding' in (layer.name.lower()):
    layer.trainable = True
    break


In [109]:
test_acc = train_deep_model(rnn_glove_200_model, X_train_seq_pad, y_train_cate, X_test_seq_pad, y_test_cate, 
                                                epochs, batch_size, lr, Lambda, callback)


Epoch 1/20
31/31 [==============================] - 194s 6s/step - loss: 2.5706 - accuracy: 0.4819 - val_loss: 6.3716 - val_accuracy: 0.0471 - lr: 0.0100
Epoch 2/20
31/31 [==============================] - 195s 6s/step - loss: 2.5166 - accuracy: 0.4863 - val_loss: 5.3921 - val_accuracy: 0.4898 - lr: 0.0100
Epoch 3/20
31/31 [==============================] - 197s 6s/step - loss: 2.5182 - accuracy: 0.4819 - val_loss: 5.0947 - val_accuracy: 0.4898 - lr: 0.0100
Epoch 4/20
31/31 [==============================] - 195s 6s/step - loss: 2.5294 - accuracy: 0.4822 - val_loss: 6.2196 - val_accuracy: 0.0277 - lr: 0.0100
Epoch 5/20
31/31 [==============================] - ETA: 0s - loss: 2.5192 - accuracy: 0.4822
Epoch 00005: ReduceLROnPlateau reducing learning rate to 0.0009999999776482583.
31/31 [==============================] - 198s 6s/step - loss: 2.5192 - accuracy: 0.4822 - val_loss: 7.3822 - val_accuracy: 0.4898 - lr: 0.0100
Epoch 6/20
31/31 [==============================] - 199s 6s/step - 

In [110]:
dl_accuracy_df = dl_accuracy_df.append(pd.Series(['RNN-Glove200-trainable-true-finetuning',
                                            accuracy_text.format(test_acc)],
                                           index=dl_accuracy_df.columns), ignore_index=True)
dl_accuracy_df


,Model,Accuracy
0,CNN-Glove200-trainable-initiallearning,Test : 46.68
1,CNN-Glove200-trainable-true-finetuning,Test : 46.31
2,CNN-SO200-trainable-initiallearning,Test : 45.57
3,CNN-SO200-trainable-true-finetuning,Test : 47.05
4,RNN-Glove-200-trainable-initiallearning,Test : 48.34
5,RNN-Glove200-trainable-true-finetuning,Test : 9.96


In [0]:
rnn_glove_200_model.save_weights(PROJECT_DIR+"weights/rnn_ownembed_200_finetuned_weights.h5")

#####Stackoverflow:

In [112]:
rnn_so_200_model = create_rnn(X_train_seq_pad.shape[1], y_train_cate.shape[1], 
                   True, vocab_size+1, embed_dim, so_embedding_matrix)

rnn_so_200_model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_16 (Embedding)     (None, 2900, 200)         589400    
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, 2900, 200)         80200     
_________________________________________________________________
simple_rnn_3 (SimpleRNN)     (None, 100)               30100     
_________________________________________________________________
dense_9 (Dense)              (None, 64)                6464      
_________________________________________________________________
batch_normalization_4 (Batch (None, 64)                256       
_________________________________________________________________
activation_4 (Activation)    (None, 64)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 52)               

In [113]:
test_acc = train_deep_model(rnn_so_200_model, X_train_seq_pad, y_train_cate, X_test_seq_pad, y_test_cate, 
                             epochs, batch_size, lr, Lambda, callback)


Epoch 1/20
31/31 [==============================] - 171s 6s/step - loss: 2.6370 - accuracy: 0.4358 - val_loss: 2.9974 - val_accuracy: 0.4918 - lr: 0.0100
Epoch 2/20
31/31 [==============================] - 170s 5s/step - loss: 2.1934 - accuracy: 0.4991 - val_loss: 2.8046 - val_accuracy: 0.4980 - lr: 0.0100
Epoch 3/20
31/31 [==============================] - 170s 5s/step - loss: 2.0292 - accuracy: 0.5183 - val_loss: 2.4731 - val_accuracy: 0.5000 - lr: 0.0100
Epoch 4/20
31/31 [==============================] - 172s 6s/step - loss: 1.9606 - accuracy: 0.5191 - val_loss: 2.6605 - val_accuracy: 0.5102 - lr: 0.0100
Epoch 5/20
31/31 [==============================] - 168s 5s/step - loss: 1.8872 - accuracy: 0.5309 - val_loss: 2.6539 - val_accuracy: 0.5174 - lr: 0.0100
Epoch 6/20
31/31 [==============================] - 168s 5s/step - loss: 1.8821 - accuracy: 0.5235 - val_loss: 2.4592 - val_accuracy: 0.5082 - lr: 0.0100
Epoch 7/20
31/31 [==============================] - 171s 6s/step - loss: 1.7

In [114]:
dl_accuracy_df = dl_accuracy_df.append(pd.Series(['RNN-SO-200-trainable-initiallearning', 
                                            accuracy_text.format(test_acc)],
                                           index=dl_accuracy_df.columns), ignore_index=True)
dl_accuracy_df

,Model,Accuracy
0,CNN-Glove200-trainable-initiallearning,Test : 46.68
1,CNN-Glove200-trainable-true-finetuning,Test : 46.31
2,CNN-SO200-trainable-initiallearning,Test : 45.57
3,CNN-SO200-trainable-true-finetuning,Test : 47.05
4,RNN-Glove-200-trainable-initiallearning,Test : 48.34
5,RNN-Glove200-trainable-true-finetuning,Test : 9.96
6,RNN-SO-200-trainable-initiallearning,Test : 47.23


In [0]:
rnn_so_200_model.save_weights(PROJECT_DIR+"weights/rnn_so_200_weights.h5")

In [0]:
for layer in rnn_so_200_model.layers:
  if 'embedding' in (layer.name.lower()):
    layer.trainable = True
    break


In [117]:
test_acc = train_deep_model(rnn_so_200_model, X_train_seq_pad, y_train_cate, X_test_seq_pad, y_test_cate, 
                                                epochs, batch_size, lr, Lambda, callback)


Epoch 1/20
31/31 [==============================] - 197s 6s/step - loss: 1.5415 - accuracy: 0.5658 - val_loss: 2.8704 - val_accuracy: 0.4836 - lr: 0.0100
Epoch 2/20
31/31 [==============================] - 198s 6s/step - loss: 1.4194 - accuracy: 0.5991 - val_loss: 3.0340 - val_accuracy: 0.5082 - lr: 0.0100
Epoch 3/20
31/31 [==============================] - 195s 6s/step - loss: 1.2341 - accuracy: 0.6365 - val_loss: 2.9086 - val_accuracy: 0.4805 - lr: 0.0100
Epoch 4/20
31/31 [==============================] - 197s 6s/step - loss: 1.1346 - accuracy: 0.6621 - val_loss: 3.0251 - val_accuracy: 0.4846 - lr: 0.0100
Epoch 5/20
31/31 [==============================] - 195s 6s/step - loss: 1.1469 - accuracy: 0.6491 - val_loss: 3.0612 - val_accuracy: 0.4785 - lr: 0.0100
Epoch 6/20
31/31 [==============================] - 196s 6s/step - loss: 1.1556 - accuracy: 0.6639 - val_loss: 3.4235 - val_accuracy: 0.4262 - lr: 0.0100
Epoch 7/20
31/31 [==============================] - ETA: 0s - loss: 1.3300 -

In [118]:
dl_accuracy_df = dl_accuracy_df.append(pd.Series(['RNN-SO-200-trainable-true-finetuning',
                                            accuracy_text.format(test_acc)],
                                           index=dl_accuracy_df.columns), ignore_index=True)
dl_accuracy_df


,Model,Accuracy
0,CNN-Glove200-trainable-initiallearning,Test : 46.68
1,CNN-Glove200-trainable-true-finetuning,Test : 46.31
2,CNN-SO200-trainable-initiallearning,Test : 45.57
3,CNN-SO200-trainable-true-finetuning,Test : 47.05
4,RNN-Glove-200-trainable-initiallearning,Test : 48.34
5,RNN-Glove200-trainable-true-finetuning,Test : 9.96
6,RNN-SO-200-trainable-initiallearning,Test : 47.23
7,RNN-SO-200-trainable-true-finetuning,Test : 44.65


In [0]:
rnn_so_200_model.save_weights(PROJECT_DIR+"weights/rnn_SO_200_finetuned_weights.h5")

####RNN - LSTM

In [0]:
def create_lstm(input_size, output_size, embedding=False, vocab_size=None, embed_dim=None, embedding_matrix=None):
    model = Sequential()
    model.add(Embedding(vocab_size, embed_dim, input_length=input_size, weights=[embedding_matrix], trainable=False))
    model.add(SpatialDropout1D(0.2))

    model.add(LSTM(200, return_sequences=True, dropout=0.2, recurrent_dropout=0.2))
    model.add(LSTM(100, return_sequences=False, dropout=0.2, recurrent_dropout=0.2))

    model.add(Dense(64, kernel_initializer='random_uniform',kernel_regularizer=regularizers.l2(Lambda)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    model.add(Dense(output_size, activation='softmax'))
    return model

#####Glove

In [42]:
lstm_glove_200_model = create_lstm(X_train_seq_pad.shape[1], y_train_cate.shape[1], 
                   True, vocab_size+1, embed_dim, glove_embedding_matrix)

lstm_glove_200_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 2900, 200)         603200    
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 2900, 200)         0         
_________________________________________________________________
lstm (LSTM)                  (None, 2900, 200)         320800    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               120400    
_________________________________________________________________
dense (Dense)                (None, 64)                6464      
_________________________________________________________________
batch_normalization (BatchNo (None, 64)                256       
_________________________________________________________________
activation (Activation)      (None, 64)                0

In [43]:
test_acc = train_deep_model(lstm_glove_200_model, X_train_seq_pad, y_train_cate, X_test_seq_pad, y_test_cate, 
                             epochs, batch_size, lr, Lambda, callback)


NameError: ignored

In [0]:
dl_accuracy_df = dl_accuracy_df.append(pd.Series(['LSTM-Glove-200-trainable-initiallearning', 
                                            accuracy_text.format(test_acc)],
                                           index=dl_accuracy_df.columns), ignore_index=True)
dl_accuracy_df

In [0]:
lstm_glove_200_model.save_weights(PROJECT_DIR+"weights/lstm_glove_200_weights.h5")

In [0]:
for layer in lstm_glove_200_model.layers:
  if 'embedding' in (layer.name.lower()):
    layer.trainable = True
    break


In [0]:
test_acc = train_deep_model(lstm_glove_200_model, X_train_seq_pad, y_train_cate, X_test_seq_pad, y_test_cate, 
                                                epochs, batch_size, lr, Lambda, callback)


In [0]:
dl_accuracy_df = dl_accuracy_df.append(pd.Series(['LSTM-Glove-200-trainable-initiallearning', 
                                            accuracy_text.format(test_acc)],
                                           index=dl_accuracy_df.columns), ignore_index=True)
dl_accuracy_df

In [0]:
lstm_glove_200_model.save_weights(PROJECT_DIR+"weights/lstm_glove_200_finetuned_weights.h5")

#####Stackoverflow

In [0]:
lstm_so_200_model = create_lstm(X_train_seq_pad.shape[1], y_train_cate.shape[1], 
                   True, vocab_size+1, embed_dim, so_embedding_matrix)

lstm_so_200_model.summary()

In [0]:
test_acc = train_deep_model(lstm_so_200_model, X_train_seq_pad, y_train_cate, X_test_seq_pad, y_test_cate, 
                             epochs, batch_size, lr, Lambda, callback)


In [0]:
dl_accuracy_df = dl_accuracy_df.append(pd.Series(['LSTM-SO-200-trainable-initiallearning', 
                                            accuracy_text.format(test_acc)],
                                           index=dl_accuracy_df.columns), ignore_index=True)
dl_accuracy_df

In [0]:
lstm_so_200_model.save_weights(PROJECT_DIR+"weights/lstm_so_200_weights.h5")

In [0]:
for layer in lstm_so_200_model.layers:
  if 'embedding' in (layer.name.lower()):
    layer.trainable = True
    break


In [0]:
test_acc = train_deep_model(lstm_so_200_model, X_train_seq_pad, y_train_cate, X_test_seq_pad, y_test_cate, 
                                                epochs, batch_size, lr, Lambda, callback)


In [0]:
dl_accuracy_df = dl_accuracy_df.append(pd.Series(['LSTM-SO-200-trainable-initiallearning', 
                                            accuracy_text.format(test_acc)],
                                           index=dl_accuracy_df.columns), ignore_index=True)
dl_accuracy_df

In [0]:
lstm_so_200_model.save_weights(PROJECT_DIR+"weights/lstm_so_200_finetuned_weights.h5")

####RNN- BiLSTM

In [0]:
def create_bi_lstm(input_size, output_size, embedding=False, vocab_size=None, embed_dim=None, embedding_matrix=None):
    model = Sequential()
    model.add(Embedding(vocab_size, embed_dim, input_length=input_size, weights=[embedding_matrix], trainable=False))

    model.add(Bidirectional(LSTM(128, return_sequences=True, recurrent_dropout=0.1, dropout=0.1))),
    model.add(Bidirectional(LSTM(64, return_sequences=False, recurrent_dropout=0.1, dropout=0.1))),

    model.add(Dense(64, kernel_initializer='random_uniform',kernel_regularizer=regularizers.l2(Lambda)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    model.add(Dense(output_size, activation='softmax'))
    return model


#####Glove

In [0]:
bi_lstm_glove_200_model = create_bi_lstm(X_train_seq_pad.shape[1], y_train_cate.shape[1], 
                   True, vocab_size+1, embed_dim, glove_embedding_matrix)

bi_lstm_glove_200_model.summary()

In [0]:
test_acc = train_deep_model(bi_lstm_glove_200_model, X_train_seq_pad, y_train_cate, X_test_seq_pad, y_test_cate, 
                             epochs, batch_size, lr, Lambda, callback)


In [0]:
dl_accuracy_df = dl_accuracy_df.append(pd.Series(['Bi-LSTM-Glove-200-trainable-initiallearning', 
                                            accuracy_text.format(test_acc)],
                                           index=dl_accuracy_df.columns), ignore_index=True)
dl_accuracy_df

In [0]:
bi_lstm_glove_200_model.save_weights(PROJECT_DIR+"weights/bi_lstm_glove_200_weights.h5")

In [0]:
for layer in bi_lstm_glove_200_model.layers:
  if 'embedding' in (layer.name.lower()):
    layer.trainable = True
    break


In [0]:
test_acc = train_deep_model(bi_lstm_glove_200_model, X_train_seq_pad, y_train_cate, X_test_seq_pad, y_test_cate, 
                                                epochs, batch_size, lr, Lambda, callback)


In [0]:
dl_accuracy_df = dl_accuracy_df.append(pd.Series(['Bi-LSTM-Glove-200-trainable-initiallearning', 
                                            accuracy_text.format(test_acc)],
                                           index=dl_accuracy_df.columns), ignore_index=True)
dl_accuracy_df

In [0]:
bi_lstm_glove_200_model.save_weights(PROJECT_DIR+"weights/bi_lstm_glove_200_finetuned_weights.h5")

#####Stackoverflow

In [0]:
bi_lstm_so_200_model = create_bi_lstm(X_train_seq_pad.shape[1], y_train_cate.shape[1], 
                   True, vocab_size+1, embed_dim, so_embedding_matrix)

bi_lstm_so_200_model.summary()

In [0]:
test_acc = train_deep_model(bi_lstm_so_200_model, X_train_seq_pad, y_train_cate, X_test_seq_pad, y_test_cate, 
                             epochs, batch_size, lr, Lambda, callback)


In [0]:
dl_accuracy_df = dl_accuracy_df.append(pd.Series(['Bi-LSTM-SO-200-trainable-initiallearning', 
                                            accuracy_text.format(test_acc)],
                                           index=dl_accuracy_df.columns), ignore_index=True)
dl_accuracy_df

In [0]:
bi_lstm_so_200_model.save_weights(PROJECT_DIR+"weights/bi_lstm_so_200_weights.h5")

In [0]:
for layer in lstm_so_200_model.layers:
  if 'embedding' in (layer.name.lower()):
    layer.trainable = True
    break


In [0]:
test_acc = train_deep_model(bi_lstm_so_200_model, X_train_seq_pad, y_train_cate, X_test_seq_pad, y_test_cate, 
                                                epochs, batch_size, lr, Lambda, callback)


In [0]:
dl_accuracy_df = dl_accuracy_df.append(pd.Series(['Bi-LSTM-SO-200-trainable-initiallearning', 
                                            accuracy_text.format(test_acc)],
                                           index=dl_accuracy_df.columns), ignore_index=True)
dl_accuracy_df

In [0]:
bi_lstm_so_200_model.save_weights(PROJECT_DIR+"weights/bi_lstm_so_200_finetuned_weights.h5")

####RNN - GRU

In [0]:
def create_rnn_gru(input_size, output_size, embedding=False, vocab_size=None, embed_dim=None, embedding_matrix=None):
    model = Sequential()
    model.add(Embedding(vocab_size, embed_dim, input_length=input_size, weights=[embedding_matrix], trainable=False))

    model.add(GRU(200, return_sequences=True, recurrent_dropout=0.1, dropout=0.1)),
    model.add(GRU(200, return_sequences=False, recurrent_dropout=0.1, dropout=0.1)),

    model.add(Dense(64, kernel_initializer='random_uniform',kernel_regularizer=regularizers.l2(Lambda)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    model.add(Dense(output_size, activation='softmax'))
    return model


#####Glove

In [0]:
rnn_gru_glove_200_model = create_rnn_gru(X_train_seq_pad.shape[1], y_train_cate.shape[1], 
                   True, vocab_size+1, embed_dim, glove_embedding_matrix)

rnn_gru_glove_200_model.summary()

In [0]:
test_acc = train_deep_model(rnn_gru_glove_200_model, X_train_seq_pad, y_train_cate, X_test_seq_pad, y_test_cate, 
                             epochs, batch_size, lr, Lambda, callback)


In [0]:
dl_accuracy_df = dl_accuracy_df.append(pd.Series(['RNN-GRU-Glove-200-trainable-initiallearning', 
                                            accuracy_text.format(test_acc)],
                                           index=dl_accuracy_df.columns), ignore_index=True)
dl_accuracy_df

In [0]:
rnn_gru_glove_200_model.save_weights(PROJECT_DIR+"weights/rnn_gru_glove_200_weights.h5")

In [0]:
for layer in rnn_gru_glove_200_model.layers:
  if 'embedding' in (layer.name.lower()):
    layer.trainable = True
    break


In [0]:
test_acc = train_deep_model(rnn_gru_glove_200_model, X_train_seq_pad, y_train_cate, X_test_seq_pad, y_test_cate, 
                                                epochs, batch_size, lr, Lambda, callback)


In [0]:
dl_accuracy_df = dl_accuracy_df.append(pd.Series(['RNN-GRU-Glove-200-trainable-initiallearning', 
                                            accuracy_text.format(test_acc)],
                                           index=dl_accuracy_df.columns), ignore_index=True)
dl_accuracy_df

In [0]:
rnn_gru_glove_200_model.save_weights(PROJECT_DIR+"weights/rnn_gru_glove_200_finetuned_weights.h5")

#####Stackoverflow

In [0]:
rnn_gru_so_200_model = create_rnn_gru(X_train_seq_pad.shape[1], y_train_cate.shape[1], 
                   True, vocab_size+1, embed_dim, so_embedding_matrix)

rnn_gru_so_200_model.summary()

In [0]:
test_acc = train_deep_model(bi_lstm_so_200_model, X_train_seq_pad, y_train_cate, X_test_seq_pad, y_test_cate, 
                             epochs, batch_size, lr, Lambda, callback)


In [0]:
dl_accuracy_df = dl_accuracy_df.append(pd.Series(['RNN-GRU-SO-200-trainable-initiallearning', 
                                            accuracy_text.format(test_acc)],
                                           index=dl_accuracy_df.columns), ignore_index=True)
dl_accuracy_df

In [0]:
rnn_gru_so_200_model.save_weights(PROJECT_DIR+"weights/rnn_gru_so_200_weights.h5")

In [0]:
for layer in rnn_gru_so_200_model.layers:
  if 'embedding' in (layer.name.lower()):
    layer.trainable = True
    break


In [0]:
test_acc = train_deep_model(rnn_gru_so_200_model, X_train_seq_pad, y_train_cate, X_test_seq_pad, y_test_cate, 
                                                epochs, batch_size, lr, Lambda, callback)


In [0]:
dl_accuracy_df = dl_accuracy_df.append(pd.Series(['RNN-GRU-SO-200-trainable-initiallearning', 
                                            accuracy_text.format(test_acc)],
                                           index=dl_accuracy_df.columns), ignore_index=True)
dl_accuracy_df

In [0]:
rnn_gru_so_200_model.save_weights(PROJECT_DIR+"weights/rnn_gru_so_200_finetuned_weights.h5")